In [ ]:
# Install required packages
%pip install numpy torch torchvision opencv-python matplotlib scikit-learn pandas albumentations PyYAML tqdm seaborn

# Training U-Net for Semantic Segmentation

This notebook demonstrates training a U-Net model for semantic segmentation on automotive video frames. It covers data loading, augmentation, model training, and evaluation. Ensure you have installed all dependencies from `requirements.txt` before running. Expected output: segmentation masks and mAP score.

# Training U-Net for Semantic Segmentation

This notebook focuses on training the U-Net model for semantic segmentation on video streams. The dataset used includes annotated video frames with segmentation masks.

In [2]:
import os
import numpy as np
import torch
from torch import nn, optim
from torchvision import transforms, datasets
from models.unet import UNetSegmenter
from src.utils import random_crop, color_jitter
from src.evaluation import calculate_mAP

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 50
learning_rate = 0.001
batch_size = 16

# Data augmentation and preprocessing
data_transforms = transforms.Compose([
    transforms.RandomApply([random_crop, color_jitter], p=0.5),
    transforms.ToTensor(),
])

# Load dataset
train_dataset = datasets.ImageFolder(root='data/processed/train', transform=data_transforms)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Initialize U-Net model
model = UNetSegmenter().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'models/unet/unet_model.pth')

# Evaluation
model.eval()
mAP = calculate_mAP(model, train_loader, device)
print(f'Mean Average Precision: {mAP:.4f}')

ModuleNotFoundError: No module named 'numpy'

## Results & Next Steps

- Review the mAP score and sample segmentation masks generated.
- For improved results, experiment with hyperparameters and data augmentation.
- Use the trained model for inference on new video frames.
- Document findings and share sample outputs in the project README.